# AI-Powered HR Assistant

## This project creates a user friendly chatbot which can answer Nestle's employees HR queries  using PDF Document. 

Tools used - <br>
    1. LangChain <br>
    2. Open AI GPT-3.5 Turbo Model <br>
    3. Gradio <br>

In [1]:
#import necessary libraries
import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import gradio as gr
from datetime import datetime, timedelta
import time


In [2]:
#Load PDF file using LangChain
def load_pdf(filename):
    pdf_loader = PyPDFLoader(filename)
    extracted_text = pdf_loader.load()
    return extracted_text

In [3]:
# Split text from documents into chunks
def recursive_charcter_text_splitter(extracted_text,chunk_size,chunk_overlap,seperator = [" "]):
    text_splitter  = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=seperator
    )
    splitted_text=text_splitter.split_documents(extracted_text)
    return splitted_text

In [4]:
#Create Embeddings using FAISS
def create_vectordb_faiss(splitted_text,embeddings):
    vectordb = FAISS.from_documents(
    documents=splitted_text,
    embedding=embeddings)
    return vectordb

In [5]:
#Create an instance of Open AI model
def create_openai_model(model_name,temperature = 0):
    llm = ChatOpenAI(model_name=model_name, temperature=temperature)
    return llm

In [6]:
#Create a chain to retreive Relavant documents based on prompts
def create_retreiver_chain(vectordb,llm,return_source_documents = False,chain_type_kwargs = None):
    retreiver_a = vectordb.as_retriever()
    retriever_chain = RetrievalQA.from_chain_type(llm,
                                           retriever =retreiver_a,
                                           return_source_documents=return_source_documents,
                                           chain_type_kwargs = chain_type_kwargs
                                           )
    return retriever_chain

In [7]:
filename = "hr_policy.pdf"
model_name = "gpt-3.5-turbo"
extracted_text = load_pdf(filename)
splitted_text = recursive_charcter_text_splitter(extracted_text,1024,256,["\n\n", "\n", "(?<=\. )", " ", ""])

In [8]:
embeddings = OpenAIEmbeddings()
vector_db = create_vectordb_faiss(splitted_text,embeddings)
llm = create_openai_model(model_name,0)
#retriever_chain = create_retreiver_chain(vector_db,llm,True,{"prompt": prompt})

In [9]:
#Creating a prompt to answer all the questions in Professsional tone using PDF as context
style = "Gujarati"
prompt_template = """You are an assistant for question-answering tasks.
Answer in a style that is """ + style + """
Question: {question}
Context: {context}
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['question'])
retriever_chain = create_retreiver_chain(vector_db,llm,True,{"prompt": prompt})


In [ ]:
#Creating function to take the questions and get the answer using GPT Model
def run_llm(message,history):
    result = retriever_chain(message)
    response = result['result']
    return response

#Creating a chat interface using Gradio
demo = gr.ChatInterface(run_llm, type="messages", autofocus=True,
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    save_history=True,
    chatbot = gr.Chatbot(placeholder="<strong>Welcome Nestle's HR BOT</strong><br>Ask Me Anything")
                       )

#Chat Interface will automatically close in 10 minutes
start_time = datetime.now()
ten_mins = 0
demo.launch(share = True)
while ten_mins == 0:
    now_time = datetime.now()
    time_difference = now_time - start_time
    nine_minutes = timedelta(minutes=10)
    if time_difference >= nine_minutes:
        demo.close()
        ten_mins = 1
    else:
        time.sleep(60)

/tmp/ipykernel_65/4169608285.py:12: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(placeholder="<strong>Welcome Nestle's HR BOT</strong><br>Ask Me Anything")
/voc/work/.local/lib/python3.10/site-packages/gradio/chat_interface.py:315: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://a1e0e54db6ed8363fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
